In [ ]:
#  The MIT License (MIT)
#
#  Copyright (c) 2015-2022 Advanced Micro Devices, Inc. All rights reserved.
#
#  Permission is hereby granted, free of charge, to any person obtaining a copy
#  of this software and associated documentation files (the 'Software'), to deal
#  in the Software without restriction, including without limitation the rights
#  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#  copies of the Software, and to permit persons to whom the Software is
#  furnished to do so, subject to the following conditions:
#
#  The above copyright notice and this permission notice shall be included in
#  all copies or substantial portions of the Software.
#
#  THE SOFTWARE IS PROVIDED 'AS IS', WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#  IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#  FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL THE
#  AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#  LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#  OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
#  THE SOFTWARE.


# Exporting Frozen Graphs in Tensorflow 2 
In order to use a trained model as input to MIGraphX, the model must be first be saved in a frozen graph format. This was accomplished in Tensorflow 1 by launching a graph in a tf.Session and then saving the session. However, Tensorflow has decided to deprecate Sessions in favor of functions and SavedModel format.  

After importing the necessary libraries, the next step is to instantiate a model. For simplicity, in this example we will use a resnet50 architecture with pre-trained imagenet weights. These weights may also be trained or fine-tuned before freezing. 

In [ ]:
import tensorflow as tf
tf.enable_eager_execution() #May not be required depending on tensorflow version
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow import keras
from tensorflow.keras import layers

MODEL_NAME = "resnet50"
model = tf.keras.applications.ResNet50(weights="imagenet")
model.summary()

## SavedModel format
The simplest way to save a model is through saved\_model.save()

This will create an equivalent tensorflow program which can later be loaded for fine-tuning or inference, although it is not directly compatible with MIGraphX.

In [ ]:
tf.saved_model.save(model, "./Saved_Models/{}".format(MODEL_NAME))

## Convert to ConcreteFunction
To begin, we need to get the function equivalent of the model and then concretize the function to avoid retracing.

In [ ]:
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

## Freeze ConcreteFunction and Serialize
Since we are saving the graph for the purpose of inference, all variables can be made constant (i.e. "frozen").

Next, we need to obtain a serialized GraphDef representation of the graph. 


Optionally, the operators can be printed out layer by layer followed by the inputs and outputs.

In [ ]:
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

## Save Frozen Graph as Protobuf
Finally, we can save to hard drive, and now the frozen graph will be stored as `./frozen_models/<MODEL_NAME>_frozen_graph.pb`

In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="{}_frozen_graph.pb".format(MODEL_NAME),
                  as_text=False)

Assuming MIGraphX has already been built and installed on your system, the driver can be used to verify that the frozen graph has been correctly exported. 

In [ ]:
import subprocess
driver = "/opt/rocm/bin/migraphx-driver"
command = "read"
model_path = "./frozen_models/{}_frozen_graph.pb".format(MODEL_NAME)
process = subprocess.run([driver, command, model_path], 
                         stdout=subprocess.PIPE, 
                         universal_newlines=True)

print(process.stdout)